## Bloque I
Descomprimir el csv en formato gz

In [1]:
""" 
Importación de ficheros de AIRBNB. Preprocesado y cargado a la base de datos 

Obtenemos los datos desde https://insideairbnb.com/get-the-data/

Podríamos hacer scraping, obtener la fecha de la última actualización de Madrid y generar el enlace para descargarlo, pero no es necesario por que se actualiza cada mucho tiempo y no es necesario tener la última versión. Podemos cambiar la info procesable en la carpeta data manualmente.

"""
import os
import pandas as pd
import numpy as np
import datetime
from config_bd import *
import requests
import sys
import gzip
import shutil

DATA_DIR = os.path.join('data')
# DATA_DIR = os.path.join(os.path.dirname(__file__), 'data')
TEMP_DIR = os.path.join(DATA_DIR, 'TEMP')


""" Comprobamos que existe el directorio de datos """

if not os.path.exists(DATA_DIR):
    print(f"El directorio {DATA_DIR} no existe. No es posible continuar...")
    sys.exit()

""" Descomprimimos el archivo de calendario """

if not os.path.exists(TEMP_DIR):
    os.makedirs(TEMP_DIR)


gz_file_path = os.path.join(DATA_DIR, 'calendar.csv.gz')
output_file_path = os.path.join(TEMP_DIR, os.path.splitext(os.path.basename(gz_file_path))[0])

if os.path.exists(gz_file_path):
    try:
        with gzip.open(gz_file_path, 'rb') as f_in:
            with open(output_file_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"Archivo {gz_file_path} descomprimido como {output_file_path}")
    except Exception as e:
        print(f"Error al descomprimir el archivo {gz_file_path}: {e}")
        sys.exit()
else:
    print(f"El archivo {gz_file_path} no existe. No es posible continuar...")
    sys.exit()


Archivo data\calendar.csv.gz descomprimido como data\TEMP\calendar.csv


## Bloque II
Carga del csv en pandas. Cambios de tipos e imputación

In [2]:
df = pd.read_csv(output_file_path, low_memory=False)

In [3]:
df['price'] = df['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
df['price'] = df['price'].fillna(0)
df['available'] = df['available'].replace({'t': 1, 'f': 0})
df['minimum_nights'] = df['minimum_nights'].fillna(df['minimum_nights'].median()).astype(int)
df['maximum_nights'] = df['maximum_nights'].fillna(df['maximum_nights'].median()).astype(int)
df = df.drop(columns=['adjusted_price'])
df = df.rename(columns={'listing_id': 'property_id'})
# df['fecha'] = pd.to_datetime(df['fecha'], format='%Y-%m-%d')

C:\Users\dsantamaria\AppData\Local\Temp\ipykernel_8836\1973130110.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['available'] = df['available'].replace({'t': 1, 'f': 0})


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
property_id,9236806.0,6.438998e+17,5.357870e+17,21853.0,36760070.0,8.115293e+17,1.142566e+18,1.369179e+18
available,9236806.0,4.376801e-01,4.961011e-01,0.0,0.0,0.000000e+00,1.000000e+00,1.000000e+00
price,9236806.0,2.116178e+02,6.396718e+02,8.0,55.0,9.200000e+01,1.600000e+02,2.100000e+04
minimum_nights,9236806.0,1.016236e+01,3.081177e+01,1.0,1.0,2.000000e+00,6.000000e+00,1.125000e+03
maximum_nights,9236806.0,6.142410e+02,4.475243e+02,1.0,364.0,3.650000e+02,1.125000e+03,1.111100e+04


In [5]:
df.sample(10)

,property_id,date,available,price,minimum_nights,maximum_nights
733547,15862047,2025-03-22,0,118.0,7,1125
2000209,34567716,2025-07-13,0,1100.0,1,4
5252906,917899379782190120,2025-08-05,0,970.0,3,365
3053031,49067339,2025-08-13,0,190.0,3,1125
1786876,31299766,2026-01-25,0,150.0,3,1125
3959800,659408045877329704,2025-10-16,1,999.0,1,365
5137344,907139239409642870,2025-03-29,0,345.0,1,364
3375111,53195801,2026-02-10,1,98.0,2,486
8496874,1303387942609921983,2025-08-17,1,90.0,1,365
3969612,666573995859381611,2025-05-06,0,120.0,3,1125


In [6]:
df.describe()

,property_id,available,price,minimum_nights,maximum_nights
count,9.236806e+06,9.236806e+06,9.236806e+06,9.236806e+06,9.236806e+06
mean,6.438998e+17,4.376801e-01,2.116178e+02,1.016236e+01,6.142410e+02
std,5.357870e+17,4.961011e-01,6.396718e+02,3.081177e+01,4.475243e+02
min,2.185300e+04,0.000000e+00,8.000000e+00,1.000000e+00,1.000000e+00
25%,3.676007e+07,0.000000e+00,5.500000e+01,1.000000e+00,3.640000e+02
50%,8.115293e+17,0.000000e+00,9.200000e+01,2.000000e+00,3.650000e+02
75%,1.142566e+18,1.000000e+00,1.600000e+02,6.000000e+00,1.125000e+03
max,1.369179e+18,1.000000e+00,2.100000e+04,1.125000e+03,1.111100e+04


## Bloque III
Importamos a la base de datos. 
**Requiere haber creado la tabla** <code>01_CREATE TABLE bnb_calendar.sql</code>

In [7]:
from config_bd import get_connection

In [8]:
df.to_csv(os.path.join(TEMP_DIR,'transform_calendar.csv'), index=False)

In [ ]:
with get_connection() as conn:            # devuelve un pyodbc.Connection
    # ——— Asegúrate de estar en autocommit para que BULK INSERT no quede en una transacción —
    original_autocommit = conn.autocommit
    conn.autocommit     = True            # activa solo durante la carga
    
    conn.execute(f"""
        BULK INSERT bnb_calendar
        FROM '{os.path.abspath(os.path.join(TEMP_DIR, 'transform_calendar.csv'))}'
        WITH (
            FORMAT='CSV',
            FIRSTROW        = 2,          -- salta cabecera
            FIELDTERMINATOR = ',',
            ROWTERMINATOR   = '0x0a',
            TABLOCK,
            BATCHSIZE       = 500000
        );
    """)
    
    conn.autocommit = original_autocommit # restaura el estado original

ProgrammingError: ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]No se puede realizar la carga masiva. El archivo 'c:\\Users\\dsantamaria\\Desktop\\TFM_GIT\\data_import\\airbnb\\data\\TEMP\\transform_calendar.csv' no se puede abrir. Código de error del sistema operativo: 3(El sistema no puede encontrar la ruta especificada.). (4861) (SQLExecDirectW)")

In [ ]:
os.remove(os.path.join(TEMP_DIR,'transform_calendar.csv'))